In [ ]:
import tensorflow as tf
import numpy as np

class LeNet:
    def __init__(self, config):
        self._num_classes = config.num_classes # label 개수 (10개-airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)
        self._l2_reg_lambda = config.l2_reg_lambda #weight decay를 위한 lamda 값

        self.X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X") # 가로: 32, 세로:32, 채널: RGB
        self.Y = tf.placeholder(tf.float32, [None, self._num_classes], name="Y") # 정답이 들어올 자리, [0 0 0 0 0 0 0 0 0 1] one-hot encoding 형태
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob") # dropout 살릴 확률
        ##############################################################################################################
        #                         TODO : LeNet5 모델 생성                                                             #
        ##############################################################################################################
        # (32, 32, 3) image
        # filter1 적용 -> (28, 28, 6) * filter1: 5*5, input_channel: 3, output_channel(# of filters): 6
        W1 = tf.Variable(tf.random_normal([5, 5, 3, 6], stddev = 0.01))
        L1 = tf.nn.conv2d(self.X, W1, strides= [1, 1, 1, 1], padding = 'VALID')
        L1 = tf.nn.relu(L1)
        L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1, 2, 2, 1], padding = 'VALID')
        # hint he initialization: stddev = sqrt(2/n), filter에서 n 값은?
        # relu -> (28, 28, 6)
        # max_pooling 적용 -> (14, 14, 6)
        # (14, 14, 6) feature map

        W2 = tf.Variable(tf.random_normal([5, 5, 6, 16], stddev = 0.01))
        L2 = tf.nn.conv2d(L1, W2, strides= [1, 1, 1, 1], padding = 'VALID')
        L2 = tf.nn.relu(L2)
        L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1, 2, 2, 1], padding = 'VALID')
        L2_flat = tf.reshape(L2, [-1, 5*5*16])
        # filter2 적용 -> (10, 10, 16) * filter1: 5*5, input_channel: 6, output_channel(# of filters): 16
        # relu -> (10, 10, 16)
        # max_pooling 적용 -> (5, 5, 16)
        # (5, 5, 16) feature map
        # 평탄화 -> (5 * 5 *16)        

        W3 = tf.get_variable('W3', shape = [5*5*16, 120], initializer = tf.random_normal_initializer(stddev = 0.01))
        b3 = tf.Variable(tf.random_normal([120]))
        L3 = tf.nn.relu(tf.matmul(L2_flat, W3) + b3)
        L3 = tf.nn.dropout(L3, keep_prob = self.keep_prob)
        # FC1 추가 (5 * 5 * 16, 120) -> (120)

        W4 = tf.get_variable('W4', shape = [120, 84])
        b4 = tf.Variable(tf.random_normal([84]))
        L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
        L4 = tf.nn.dropout(L4, keep_prob = self.keep_prob)

        W5 = tf.get_variable('W5', shape = [84, 10], initializer = tf.random_normal_initializer(stddev = 0.01))
        b5 = tf.Variable(tf.random_normal([10]))
        # (120) features
        # FC2 추가 (120, 84) -> (84)
        # (84) features
        # Softmax layer 추가 (84) -> (10)
        hypothesis = tf.nn.xw_plus_b(L4, W5, b5, name="hypothesis") #hypothesis를 softmax를 통해 확률값으로 변형, Y와 비교해 cross_entropy error 계산


        with tf.variable_scope('logit'):
            self.predictions = tf.argmax(hypothesis, 1, name="predictions")

        with tf.variable_scope('loss'):
            costs = []
            for var in tf.trainable_variables():
                costs.append(tf.nn.l2_loss(var)) # 모든 가중치들의 l2_loss 누적
            l2_loss = tf.add_n(costs)
            xent = tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=self.Y)
            self.loss = tf.reduce_mean(xent, name='xent') + self._l2_reg_lambda * l2_loss

        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

# New Section